In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

pd.options.display.max_colwidth = 200

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:2]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install librosa
# !pip install SpeechRecognition
# !pip install googletrans
# !pip install gTTs
# !pip install PyAudio

#!pip install googletrans==3.1.0a0
#!pip install googletrans==4.0.0-rc1

In [ ]:
import librosa
import IPython.display as ipd
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS

# <center> Speech to Text 

## Speech Recognition Functions

we will see our recognizer in action but before we get it to work let’s see some cool functions of this instance. Speech Recognition has a built-in function to make it work with many of the APIs out there:
    
* recognize_bing()
* recognize_google()
* recognize_google_cloud()
* recognize_wit()

Bing Recognizer function uses Microsoft’s cognitive services. <br>

Google Recognizer function uses Google’s free web search API. <br>

Google Cloud Recognizer function uses Google’s cloud speech API. <br>

Wit Recognizer function uses the wit.ai platform. <br>

We will use the Google Recognizer function, which is __recognize_google()__. It’s free and doesn’t require an API key to use. There is one drawback about this recognizer, it limits you when you want to work with longer audio files. In my experience, I didn’t have any issues when working with audio files under 5 minutes. I don’t recommend using this recognizer with long audio files. There are different techniques to work with longer audio files.

In [ ]:
ipd.Audio('/kaggle/input/nlp-specialization-data/harvard.wav')

In [ ]:
ipd.Audio('/kaggle/input/nlp-specialization-data/testSpeech.wav')

In [ ]:
ipd.Audio('/kaggle/input/nlp-specialization-data/singleEnglishWord.wav')

In [ ]:
r = sr.Recognizer()

In [ ]:
harvard = sr.AudioFile('/kaggle/input/nlp-specialization-data/harvard.wav')
with harvard as source:
    audio = r.record(source)

In [ ]:
type(audio)

In [ ]:
text=r.recognize_google(audio)
text

**Note:**

* The __offset__ and __duration__ keyword arguments are useful for segmenting an audio file if you have prior knowledge of the structure of the speech in the file.

* Offset For instance, if you do not want to transcribe the first 4 seconds of the audio, pass 4 as the value for the offset attribute. As an example, the following script skips the first 4 seconds of the audio file

In [ ]:
with harvard as source:
    audio = r.record(source, offset=4)
r.recognize_google(audio)

In [ ]:
with harvard as source:
    audio = r.record(source, duration=3)
r.recognize_google(audio)

In [ ]:
with harvard as source:
    audio = r.record(source, offset=4, duration=5)
r.recognize_google(audio)

# <center> Speech to Speech

In [ ]:
#!pip install google_trans_new

from google_trans_new import google_translator  
translator = google_translator()  
translate_text = translator.translate("hello", lang_src='en',lang_tgt='hi')  
print(translate_text)
#output: Hello china

In [ ]:
r = sr.Recognizer()
harvard = sr.AudioFile('/kaggle/input/nlp-specialization-data/harvard.wav')
with harvard as source:
    audio = r.record(source)
text=r.recognize_google(audio)
text

In [ ]:
translatorInstance=Translator()
translatedText = translatorInstance.translate(text,dest='hi')
print(translatedText.text.split(' '))

In [ ]:
# mytext = translator.translate(text,dest='la').text
language = 'hi'
translatedObject= gTTS(text=translatedText.text, lang=language, slow=False)
translatedObject.save('translatedAudio.mp3') 

In [ ]:
import IPython.display as ipd
ipd.Audio('translatedAudio.mp3')

In [ ]:
translatedText = translatorInstance.translate(text,dest='ur').text.split(' ') 
print(translatedText)

In [ ]:
translatedText = translatorInstance.translate(text,dest='es').text.split(' ') 
print(translatedText)

### Try Yourself Now! 

In [ ]:
def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.
    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source) # #  analyze the audio source for 1 second
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #   update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable/unresponsive"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response

In [ ]:
if __name__ == "__main__":
    recognizer = sr.Recognizer()
    mic = sr.Microphone(device_index=1)
    response = recognize_speech_from_mic(recognizer, mic)
    print('\nSuccess : {}\nError   : {}\n\nText from Speech\n{}\n\n{}' \
          .format(response['success'],
                  response['error'],
                  '-'*17,
                  response['transcription']))

## <center> Bonus

### 1. Speechless Audio

In [ ]:
# Import the silent audio file
silent_audio_file = sr.AudioFile("silent_audio.wav")
# Convert the AudioFile to AudioData
with silent_audio_file as source:
    silent_audio = recognizer.record(source)
# Recognize the AudioData with show_all turned on
recognizer.recognize_google(silent_audio, show_all=True)

### 2. Multiple Speakers
This process of understanding the different speakers in a single audio file is known as speaker diarization. This is a really cool function to have but unfortunately, it is not available in this library. One solution to do this is to have different audio files for different speakers, go through them using for loop.

In [ ]:
recognizer = sr.Recognizer()
# Multiple speakers on different files
speakers = [sr.AudioFile("speaker_0.wav"), sr.AudioFile("speaker_1.wav"), sr.AudioFile("speaker_2.wav")]
# Transcribe each speaker individually
for i, speaker in enumerate(speakers):
    with speaker as source:
    speaker_audio = recognizer.record(source)
print(f"Text from speaker {i}:")
print(recognizer.recognize_google(speaker_audio,language="en-US"))

### 3. Background Noise
To handle the background noise, the recognizer class has a built-in function called adjust_for_ambient_noise function, which also takes a parameter of duration. Using this function the recognizer class listens to the audio for the specified duration seconds from the beginning of the audio and then adjusts the energy threshold value so that the whole audio is more recognizable.

In [ ]:
# Import audio file with background nosie
noisy_audio_file = sr.AudioFile("noisy_audio.wav")
# Adjust for ambient noise and record
with noisy_audio_file as source:
    recognizer.adjust_for_ambient_noise(source, duration=0.5)
    noisy_audio_file = recognizer.record(source)
# Recognize the audio
recognizer.recognize_google(noisy_audio_file)

## Reference Links 

* __Learn how to Build your own Speech-to-Text Model (using Python) -__ www.analyticsvidhya.com/blog/2019/07/learn-build-first-speech-to-text-model-python/